# Using Externals Tools

In this notebook we are going to show you how use the `FunctionTool` to bind a python function to a tool for the Agent system to use.
Because we cannot build every tool that people want, we provide an easy way for you to implement them.

In [1]:
import requests
from hybridagi.modules.agents.tools import FunctionTool

def get_crypto_price(crypto_name: str):
    """
    Please only give the name of the crypto to fetch like "bitcoin" or "cardano"
    Never explain or apology, only give the crypto name.
    """
    base_url = "https://api.coingecko.com/api/v3/simple/price?"
    complete_url = base_url + "ids=" + crypto_name + "&vs_currencies=usd"
    response = requests.get(complete_url)
    data = response.json()

    if crypto_name in data:
        return {"crypto_name": crypto_name, "result": str(data[crypto_name]["usd"])+" USD"}
    else:
        return {"crypto_name": crypto_name, "result": "Invalid crypto name"}
    
my_tool = FunctionTool(
    name = "GetCryptoPrice",
    func = get_crypto_price,
)

/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import hybridagi.core.graph_program as gp

main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Decision(
    id = "is_websearch_needed",
    purpose = "Check if the Objective's question is about getting the crypto prices",
    question = "Is the Objective's question about crypto prices?",
))

main.add(gp.Action(
    id = "fetch_prices",
    purpose = "Fetch the price of a crypto currency",
    tool = "GetCryptoPrice",
    prompt = "Use the Objective's question to infer the search query",
))

main.add(gp.Action(
    id = "answer",
    purpose = "Answer the Objective's question",
    tool = "Speak",
    prompt = "Answer the Objective's question, use your context if relevant",
))

main.connect("start", "is_websearch_needed")
main.connect("is_websearch_needed", "fetch_prices", label="Fetch prices")
main.connect("is_websearch_needed", "answer", label="Answer")
main.connect("fetch_prices", "answer")
main.connect("answer", "end")

main.build()

print(main)

// @desc: The main program
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(is_websearch_needed:Decision {
  id: "is_websearch_needed",
  purpose: "Check if the Objective's question is about getting the crypto prices",
  question: "Is the Objective's question about crypto prices?"
}),
(fetch_prices:Action {
  id: "fetch_prices",
  purpose: "Fetch the price of a crypto currency",
  tool: "GetCryptoPrice",
  prompt: "Use the Objective's question to infer the search query"
}),
(answer:Action {
  id: "answer",
  purpose: "Answer the Objective's question",
  tool: "Speak",
  prompt: "Answer the Objective's question, use your context if relevant"
}),
// Structure declaration
(start)-[:NEXT]->(is_websearch_needed),
(is_websearch_needed)-[:FETCH_PRICES]->(fetch_prices),
(is_websearch_needed)-[:ANSWER]->(answer),
(fetch_prices)-[:NEXT]->(answer),
(answer)-[:NEXT]->(end)


In [3]:
# We load our program into memory

from hybridagi.memory.integration.local import LocalProgramMemory

program_memory = LocalProgramMemory(index_name="custom_tool")

program_memory.update(main)

In [4]:
import dspy
from hybridagi.core.datatypes import AgentState, Query
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.modules.agents.tools import (
    SpeakTool,
)

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state
    ),
    my_tool,
]

rag_agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
    debug = True,
)

# We can now setup the LLM using Ollama client from DSPy

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.configure(lm=lm)

# And call our agent

result = rag_agent(Query(query="What is the price of ethereum?"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: What is the price of ethereum?
--- Step 1 ---
Decision Purpose: Check if the Objective's question is about getting the crypto prices
Decision Question: Is the Objective's question about crypto prices?
Decision: FETCH_PRICES
--- Step 2 ---
Action Purpose: Fetch the price of a crypto currency
Action: {
  "crypto_name": "ethereum",
  "result": "2467.57 USD"
}
--- Step 3 ---
Action Purpose: Answer the Objective's question
Action: {
  "message": "The price of Ethereum is currently 2467.57 USD."
}
--- Step 4 ---
End Program: main
The price of Ethereum is currently 2467.57 USD.
